## Configuração do ambiente para utilização do Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession

# Criando a sessão do Spark com configuração de memória aumentada
spark = SparkSession.builder \
    .appName("Feature Engineering") \
    .config("spark.executor.memory", "40g") \
    .config("spark.driver.memory", "40g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.cores", "4") \
    .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
# Lendo arquivo csv
dados = spark.read.csv("PoD Bank/bureau.csv",header=True)


In [ ]:
## Habilitando uso do SparkSQL
dados.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN DAYS_CREDIT >= -90 THEN 1
        ELSE 0
    END AS ultimos_3_meses,
    CASE
        WHEN DAYS_CREDIT >= -180 THEN 1
        ELSE 0
    END AS ultimos_6_meses,
    CASE
        WHEN DAYS_CREDIT >= -360 THEN 1
        ELSE 0
    END AS ultimos_12_meses,
    CASE
        WHEN DAYS_CREDIT >= -540 THEN 1
        ELSE 0
    END AS ultimos_18_meses,
    CASE
        WHEN DAYS_CREDIT >= -720 THEN 1
        ELSE 0
    END AS ultimos_24_meses,
    CASE
        WHEN DAYS_CREDIT >= -900 THEN 1
        ELSE 0
    END AS ultimos_30_meses,
    CASE
        WHEN DAYS_CREDIT >= -1080 THEN 1
        ELSE 0
    END AS ultimos_36_meses
FROM dados
ORDER BY `SK_ID_BUREAU`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")

In [ ]:
df_temp_01.createOrReplaceTempView("df_temp_01")

df_temp_02 = spark.sql("""
SELECT
    *,
      CASE
        WHEN CREDIT_ACTIVE = "Closed" THEN 1
        ELSE 0
    END AS CREDIT_ACTIVE_CLOSED,
    CASE
        WHEN CREDIT_ACTIVE = "Active" THEN 1
        ELSE 0
    END AS CREDIT_ACTIVE_ACTIVE,
    CASE
        WHEN CREDIT_ACTIVE = "Sold" THEN 1
        ELSE 0
    END AS CREDIT_ACTIVE_SOLD,
    CASE
        WHEN CREDIT_ACTIVE = "Bad debt" THEN 1
        ELSE 0
    END AS CREDIT_ACTIVE_BAD_DEBT
FROM df_temp_01
ORDER BY `SK_ID_BUREAU`;
""")
df_temp_02.createOrReplaceTempView("df_temp_01")


In [ ]:
df_temp_02.createOrReplaceTempView("df_temp_02")

df_temp_03 = spark.sql("""
SELECT
    *,
      CASE
        WHEN CREDIT_CURRENCY = "currency 1" THEN 1
        ELSE 0
    END AS CREDIT_CURRENCY_currency_1,
    CASE
        WHEN CREDIT_CURRENCY = "currency 2" THEN 1
        ELSE 0
    END AS CREDIT_CURRENCY_currency_2,
    CASE
        WHEN CREDIT_CURRENCY = "currency 3" THEN 1
        ELSE 0
    END AS CREDIT_CURRENCY_currency_3,
    CASE
        WHEN CREDIT_CURRENCY = "currency 4" THEN 1
        ELSE 0
    END AS CREDIT_CURRENCY_currency_4
FROM df_temp_02
ORDER BY `SK_ID_BUREAU`;
""")
df_temp_03.createOrReplaceTempView("df_temp_02")


## Sumarizar na visão cliente (Automatizada)

In [ ]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when

# Função para gerar expressões de agregação com flags
def gerar_expressoes_agregacao_com_flags(colunas_agregacao_total, colunas_flags):
    expressoes_agregacao = []

    for flag in colunas_flags:
        for coluna in colunas_agregacao_total:
            if 'DAY' in coluna:
                expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}"))
                expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}"))
            else:
                expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}"))
                expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}"))
                expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}"))
                expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}"))

    return expressoes_agregacao

# Função para gerar expressões de agregação sem flags, apenas com categorias
def gerar_expressoes_agregacao_sem_flags(colunas_agregacao_total, colunas_cat):
    expressoes_agregacao = []

    for cat in colunas_cat:
        for coluna in colunas_agregacao_total:
            if 'DAY' in coluna:
                expressoes_agregacao.append(round(max(when(col(cat) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{cat.upper()}"))
                expressoes_agregacao.append(round(min(when(col(cat) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{cat.upper()}"))
            else:
                expressoes_agregacao.append(round(sum(when(col(cat) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{cat.upper()}"))
                expressoes_agregacao.append(round(avg(when(col(cat) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{cat.upper()}"))
                expressoes_agregacao.append(round(max(when(col(cat) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{cat.upper()}"))
                expressoes_agregacao.append(round(min(when(col(cat) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{cat.upper()}"))

    return expressoes_agregacao

# Definir as colunas para agregação
colunas_agregacao_total = ['CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE',
                           'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
                           'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']

colunas_flags = ['ultimos_3_meses','ultimos_6_meses', 'ultimos_12_meses', 'ultimos_18_meses',  'ultimos_24_meses',  'ultimos_30_meses',
'ultimos_36_meses']
colunas_cat1 = ['CREDIT_ACTIVE_CLOSED', 'CREDIT_ACTIVE_ACTIVE', 'CREDIT_ACTIVE_SOLD', 'CREDIT_ACTIVE_BAD_DEBT']
colunas_cat2 = ['CREDIT_CURRENCY_currency_1', 'CREDIT_CURRENCY_currency_2', 'CREDIT_CURRENCY_currency_3', 'CREDIT_CURRENCY_currency_4']

# Gerar expressões de agregação para flags
expressoes_agregacao_flags = gerar_expressoes_agregacao_com_flags(colunas_agregacao_total, colunas_flags)

# Gerar expressões de agregação para categorias (sem flags)
expressoes_agregacao_cat1 = gerar_expressoes_agregacao_sem_flags(colunas_agregacao_total, colunas_cat1)
expressoes_agregacao_cat2 = gerar_expressoes_agregacao_sem_flags(colunas_agregacao_total, colunas_cat2)

# Combinar todas as expressões de agregação
expressoes_agregacao = expressoes_agregacao_flags + expressoes_agregacao_cat1 + expressoes_agregacao_cat2

# Aplicar as expressões de agregação
df_temp_04 = df_temp_03.groupBy("SK_ID_BUREAU").agg(*expressoes_agregacao).orderBy("SK_ID_BUREAU")



## Join com a bureau balance

In [ ]:
bureau_balance = spark.read.parquet("PoD Bank/Tabelas - Feature_Engineering/bureau_balance.parquet")

In [ ]:
df_temp_05 = df_temp_04.join(bureau_balance, on='SK_ID_BUREAU', how='left')


## Join deste resultado a base bureau

In [ ]:
df_temp_06 = df_temp_03.select("SK_ID_BUREAU", "SK_ID_CURR").join(df_temp_05, on='SK_ID_BUREAU', how='left')


## Sumarizar base resultante na visão cliente

In [ ]:
# Definir as colunas para agregação
colunas_agregacao_total = df_temp_06.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_BUREAU')

expressoes_agregacao = []

for coluna in colunas_agregacao_total:
  if 'DAY' in coluna:
    expressoes_agregacao.append(round(max(col(coluna)), 2).alias(f"QT_MAX_{coluna.upper()}"))
    expressoes_agregacao.append(round(min(col(coluna)), 2).alias(f"QT_MIN_{coluna.upper()}"))
  else:
    expressoes_agregacao.append(round(sum(col(coluna)), 2).alias(f"VL_TOT_{coluna.upper()}"))
    expressoes_agregacao.append(round(avg(col(coluna)), 2).alias(f"VL_MED_{coluna.upper()}"))
    expressoes_agregacao.append(round(max(col(coluna)), 2).alias(f"VL_MAX_{coluna.upper()}"))
    expressoes_agregacao.append(round(min(col(coluna)), 2).alias(f"VL_MIN_{coluna.upper()}"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_07 = df_temp_06.groupBy("SK_ID_CURR").agg(*expressoes_agregacao).orderBy("SK_ID_CURR")



In [ ]:
df_temp_07 = df_temp_07.repartition(1)
df_temp_07.write.mode("overwrite").parquet("PoD Bank/Tabelas - Feature_Engineering/bureau_agg")